In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
no_requirements_responses = pd.read_csv(r'C:\Users\Will\Files\Florida\Research\NLP\Mechanical Turk Datasets\No Requirements - Drug Discontinuation 1__Batch_4794452_batch_results.csv')
greater_than_85_percent_acceptance_responses = pd.read_csv(r'C:\Users\Will\Files\Florida\Research\NLP\Mechanical Turk Datasets\greater than 85 percent satisfaction - Drug Discontinuation 4__Batch_4794448_batch_results.csv')
greater_than_95_percent_acceptance_responses = pd.read_csv(r'C:\Users\Will\Files\GitHub_Projects\medhelp.org_drug_discontinuation_dataset\Data _From_Mechanical_Turk\greater than 95 - 5 labellers per comment - Batch_4795462_batch_results.csv')

In [3]:
data1 = no_requirements_responses
data2 = greater_than_85_percent_acceptance_responses
data3 = greater_than_95_percent_acceptance_responses

In [4]:
data1_n_labellers_per_comment = 3
data2_n_labellers_per_comment = 3
data3_n_labellers_per_comment = 5

In [5]:
# data1.head()
data1.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.m_1',
       'Input.url', 'Input.text_', 'Input.text', 'Answer.taskAnswers',
       'Approve', 'Reject'],
      dtype='object')

In [5]:
data1['Input.m_1'] = data1['Input.m_1'].str.split(' ').str[0]
data2['Input.m_1'] = data2['Input.m_1'].str.split(' ').str[0]
data3['Input.m_1'] = data3['Input.m_1'].str.split(' ').str[0]

In [7]:
data1['Input.m_1'].head()

0    No
1    No
2    No
3    No
4    No
Name: Input.m_1, dtype: object

In [8]:
data1['Answer.taskAnswers'].head()

0    [{"q1a":{"no":false,"yes":true},"q1b":{"not_co...
1    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
2    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
3    [{"q1a":{"no":false,"yes":true},"q1b":{"not_co...
4    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
Name: Answer.taskAnswers, dtype: object

In [9]:
data1['Answer.taskAnswers'][0]

'[{"q1a":{"no":false,"yes":true},"q1b":{"not_confident":false,"somewhat_confident":false,"very_confident":true},"q2a":{"q2a1":false,"q2a2":false,"q2a3":false,"q2a4":false,"q2a5":true,"q2a6":false,"q2a7":false,"q2a8":false},"q2b":{"not_confident":true,"somewhat_confident":false,"very_confident":false}}]'

In [6]:
data1 = data1.join(data1['Answer.taskAnswers'].apply(json.loads).apply(pd.Series))
data2 = data2.join(data2['Answer.taskAnswers'].apply(json.loads).apply(pd.Series))
data3 = data3.join(data3['Answer.taskAnswers'].apply(json.loads).apply(pd.Series))

In [11]:
data1.columns

Index([                      'HITId',                   'HITTypeId',
                             'Title',                 'Description',
                          'Keywords',                      'Reward',
                      'CreationTime',              'MaxAssignments',
               'RequesterAnnotation', 'AssignmentDurationInSeconds',
        'AutoApprovalDelayInSeconds',                  'Expiration',
               'NumberOfSimilarHITs',           'LifetimeInSeconds',
                      'AssignmentId',                    'WorkerId',
                  'AssignmentStatus',                  'AcceptTime',
                        'SubmitTime',            'AutoApprovalTime',
                      'ApprovalTime',               'RejectionTime',
                 'RequesterFeedback',           'WorkTimeInSeconds',
              'LifetimeApprovalRate',      'Last30DaysApprovalRate',
             'Last7DaysApprovalRate',                   'Input.m_1',
                         'Input.ur

In [12]:
data1[0].head()

0    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
1    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
2    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
3    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
4    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
Name: 0, dtype: object

In [7]:
def get_label(row):
    if row[0]['q1a']['no']:
        return 'No'
    else:
        return 'Yes'
data1['label'] = data1.apply(get_label, axis=1)
data2['label'] = data2.apply(get_label, axis=1)
data3['label'] = data3.apply(get_label, axis=1)

data1['label'].head()

0    Yes
1     No
2     No
3    Yes
4     No
Name: label, dtype: object

In [8]:
def get_confidence(row):    
    if row[0]['q1b']['very_confident']:
        return 'very_confident'
    elif row[0]['q1b']['somewhat_confident']:
        return 'somewhat_confident'
    else:
        return 'not_confident'
data1['confidence'] = data1.apply(get_confidence, axis=1)
data2['confidence'] = data2.apply(get_confidence, axis=1)
data3['confidence'] = data3.apply(get_confidence, axis=1)

data1['confidence'].head()

0        very_confident
1        very_confident
2        very_confident
3        very_confident
4    somewhat_confident
Name: confidence, dtype: object

In [9]:
# data1['correct_label'] = np.where(data1[0]['q1a'] data1['Answer.taskAnswers'].apply(lambda x: x[0]['correct_label']), 1, 0)
data1['correct_label'] = (data1['label']==data1['Input.m_1'])
data2['correct_label'] = (data2['label']==data2['Input.m_1'])
data3['correct_label'] = (data3['label']==data3['Input.m_1'])
data1['correct_label'].head()

0    False
1     True
2     True
3    False
4     True
Name: correct_label, dtype: bool

In [11]:
print(data1['correct_label'].value_counts())
print(data2['correct_label'].value_counts())
print(data3['correct_label'].value_counts())
print(len(data3))
print(data3['correct_label'].value_counts()[1] /len(data3))

True     396
False    252
Name: correct_label, dtype: int64
True     352
False    296
Name: correct_label, dtype: int64
True     634
False    446
Name: correct_label, dtype: int64
1080
0.587037037037037


In [16]:
n_data_points = len(data1)
print(n_data_points)

648


In [10]:
a = data1['correct_label'].value_counts()
b = data2['correct_label'].value_counts()
c = data3['correct_label'].value_counts()

print(a[1]/n_data_points)
print(b[1]/n_data_points)
print(c[1]/len(data3))

NameError: name 'n_data_points' is not defined

In [24]:
print(data1[0].head())

0    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
1    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
2    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
3    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
4    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
Name: 0, dtype: object


In [11]:
#"q1b":{"not_confident":false,"somewhat_confident":false,"very_confident":true}

data1_very_confident = data1.loc[data1['confidence']== 'very_confident']
a = data1_very_confident['correct_label'].value_counts()
print(len(data1_very_confident))
print(a)
print(a[1] / len(data1_very_confident))

data2_very_confident = data2.loc[data2['confidence']=='very_confident']
b = data2_very_confident['correct_label'].value_counts()
print(len(data2_very_confident))
print(b)
print(b[1] / len(data2_very_confident))

data3_very_confident = data3.loc[data3['confidence']=='very_confident']
c = data3_very_confident['correct_label'].value_counts()
print(len(data3_very_confident))
print(c)
print(c[1] / len(data3_very_confident))
# print(data2_very_confident['correct_label'].value_counts())

384
True     259
False    125
Name: correct_label, dtype: int64
0.6744791666666666
318
True     213
False    105
Name: correct_label, dtype: int64
0.6698113207547169
519
True     378
False    141
Name: correct_label, dtype: int64
0.7283236994219653


In [12]:
data1_somewhat_confident = data1.loc[data1['confidence']== 'somewhat_confident']
a = data1_somewhat_confident['correct_label'].value_counts()
print(len(data1_somewhat_confident))
print(a)
print(a[1] / len(data1_somewhat_confident))
data2_somewhat_confident = data2.loc[data2['confidence']=='somewhat_confident']
b = data2_somewhat_confident['correct_label'].value_counts()
print(len(data2_somewhat_confident))
print(b)
print(b[1] / len(data2_somewhat_confident))
data3_somewhat_confident = data3.loc[data3['confidence']=='somewhat_confident']
c = data3_somewhat_confident['correct_label'].value_counts()
print(len(data3_somewhat_confident))
print(c)
print(c[1] / len(data3_somewhat_confident))

245
False    124
True     121
Name: correct_label, dtype: int64
0.49387755102040815
292
False    178
True     114
Name: correct_label, dtype: int64
0.3904109589041096
528
False    291
True     237
Name: correct_label, dtype: int64
0.44886363636363635


In [13]:
data1_not_confident = data1.loc[data1['confidence']== 'not_confident']
a = data1_not_confident['correct_label'].value_counts()
print(len(data1_not_confident))
print(a)
print(a[1] / len(data1_not_confident))
data2_not_confident = data2.loc[data2['confidence']=='not_confident']
b = data2_not_confident['correct_label'].value_counts()
print(len(data2_not_confident))
print(b)
print(b[1] / len(data2_not_confident))
data3_not_confident = data3.loc[data3['confidence']=='not_confident']
c = data3_not_confident['correct_label'].value_counts()
print(len(data3_not_confident))
print(c)
print(c[1] / len(data3_not_confident))

19
True     16
False     3
Name: correct_label, dtype: int64
0.8421052631578947
38
True     25
False    13
Name: correct_label, dtype: int64
0.6578947368421053
33
True     19
False    14
Name: correct_label, dtype: int64
0.5757575757575758


In [14]:
# Compare the accuracy of the "Yes" and "No" labels
data1_no = data1.loc[data1['label']=='Yes']
a = data1_no['correct_label'].value_counts()
print(len(data1_yes))
print(a)
print(a[1] / len(data1_yes))
data2_yes = data2.loc[data2['label']=='Yes']
b = data2_yes['correct_label'].value_counts()
print(len(data2_yes))
print(b)
print(b[1] / len(data2_yes))
data3_yes = data3.loc[data3['label']=='Yes']
c = data3_yes['correct_label'].value_counts()
print(len(data3_yes))
print(c)
print(c[1] / len(data3_yes))

print("\nNOW FOR NO\n")
data1_no = data1.loc[data1['label']=='No']
a = data1_no['correct_label'].value_counts()
print(len(data1_no))
print(a)
print(a[1] / len(data1_no))
data2_no = data2.loc[data2['label']=='No']
b = data2_no['correct_label'].value_counts()
print(len(data2_no))
print(b)
print(b[1] / len(data2_no))
data3_no = data3.loc[data3['label']=='No']
c = data3_no['correct_label'].value_counts()
print(len(data3_no))
print(c)
print(c[1] / len(data3_no))



373
False    203
True     170
Name: correct_label, dtype: int64
0.45576407506702415
457
False    267
True     190
Name: correct_label, dtype: int64
0.41575492341356673
719
False    400
True     319
Name: correct_label, dtype: int64
0.44367176634214184

NOW FOR NO

275
True     226
False     49
Name: correct_label, dtype: int64
0.8218181818181818
191
True     162
False     29
Name: correct_label, dtype: int64
0.8481675392670157
361
True     315
False     46
Name: correct_label, dtype: int64
0.8725761772853186


In [26]:
data1['confidence']

0      Yes
1       No
2       No
3      Yes
4       No
      ... 
643     No
644    Yes
645     No
646     No
647     No
Name: confidence, Length: 648, dtype: object

In [ ]:
# Exclude labels which had a low confidence

# Exclude labels which all workers didn't agree on


In [66]:
print(data1['WorkerId'].value_counts())
print(data2['WorkerId'].value_counts())
print(data3['WorkerId'].value_counts())

A29HILIOBLRW1S    47
A2EYD5BP3AKLIE    37
A64F60OM8VDDQ     33
AFA73M8WJKEN0     29
ACU8RTAZQU1GP     28
                  ..
A2N8CRZSSKEDH3     1
A1FDCIC289ILHM     1
AZHNR434Q0PCF      1
ASXWSNSIHXAHT      1
AV6C736XAEG67      1
Name: WorkerId, Length: 111, dtype: int64
A3UBH7E0BEB23K    73
AXJCV4FWC4BGQ     36
APXN6ZCXRGB5M     33
A23LU9N9IT220U    32
A19940RY4CF491    23
                  ..
A3EJ9921TCZUEJ     1
A33SZTK9OMONNB     1
A3T467USWU7ZOY     1
A10NF5TK6IFNX6     1
A3335ASFR3L0Z8     1
Name: WorkerId, Length: 133, dtype: int64
A2MI7L5QRNPV1Y    115
AN0FKN6TFVP2D      77
AUFQA9PSISZOX      64
AXJCV4FWC4BGQ      50
A2EYD5BP3AKLIE     48
                 ... 
A2E681TUBSHLWD      1
A1J3NRLGFEIS5H      1
A2QX3YJXAAHHVV      1
A36QGCT3MMXC4Q      1
AJQGWGESKQT4Y       1
Name: WorkerId, Length: 136, dtype: int64


In [67]:
print(set(data1['WorkerId'].unique()).intersection(set(data2['WorkerId'].unique())))
print(len(set(data1['WorkerId'].unique()).intersection(set(data2['WorkerId'].unique()))))
print(len(set(data1['WorkerId'].unique()).intersection(set(data3['WorkerId'].unique()))))
print(len(set(data2['WorkerId'].unique()).intersection(set(data3['WorkerId'].unique()))))
# print(data2['WorkerId'].value_counts())

{'ABDXLB3YCM6XY', 'A14CCWIFLDPZLA', 'AFA73M8WJKEN0', 'A3OTX62IHUFV3D', 'A2E6Z6NDAES26P', 'AXJCV4FWC4BGQ', 'A3MHMMTUTOJQ1V', 'A26EEQE7NYLQNI', 'A1CZZ8E7K3CCV9', 'A27JUKQFY00XP7', 'A3ZSLY8UQDRC3', 'A1K0LRUQUN5I0J', 'A2WYZLG8J526TQ', 'APGOZK7LO6YQF', 'A2VUF0V7HT51Q3', 'A3A6BVDDO2M1C9', 'A5J3EJX1ADIS', 'AMOBC1D09Q381', 'A38AQF5BPHJS4C', 'A1MDKN4ZIZYH2O', 'A2V40CL511Z37G', 'A1LJRMT897VBKP', 'A2ZLJQWCM8KU36', 'A1XHSOHTQ81V4O', 'A3T467USWU7ZOY', 'A2J9NFY45NKR2N', 'A2EYD5BP3AKLIE', 'A29HILIOBLRW1S', 'ARPBDM5QZ4XQC', 'A23LU9N9IT220U', 'A1HEB7F1J8ENZI', 'A2ZT2Z0LQ05EGZ', 'A38ZFP228C5112', 'AV6C736XAEG67', 'ADVCIFLB5A9B', 'A30LVINW75GV0V', 'A2N8CRZSSKEDH3', 'A3EM9OBO5KU9GG', 'A2LBYE8RUQBQYG'}
39
8
11


In [17]:
# Find the number of comments in the data that all labelers agreed on
a1 = data1.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a2 = data2.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a3 = data3.groupby(['Input.url']).label.nunique().eq(1).value_counts()
print(a1)
print(a1[1]/(n_data_points/3))
print(a2)
print(a2[1]/(n_data_points/3))
print(a3)
print(a3[1]/(n_data_points/3))



False    137
True      78
Name: label, dtype: int64
0.3611111111111111
False    119
True      96
Name: label, dtype: int64
0.4444444444444444
False    165
True      50
Name: label, dtype: int64
0.23148148148148148


In [31]:
data1['label'].iloc[0]

'Yes'

In [71]:
# get the subset of data that all labelers agreed on for each dataset

data1_unan_urls = []
data2_unan_urls = []
data3_unan_urls = []

data1_yes_urls = []
data2_yes_urls = []
data3_yes_urls = []

data1_no_urls = []
data2_no_urls = []
data3_no_urls = []

for url in data1['Input.url'].unique():
    if data1.groupby(['Input.url']).label.nunique().eq(1).loc[url] == True:
        data1_unan_urls.append(url)
        if data1.loc[data1['Input.url']==url]['label'].iloc[0] =="Yes":
            data1_yes_urls.append(url)
        elif data1.loc[data1['Input.url']==url]['label'].iloc[0] =="No":
            data1_no_urls.append(url)
    if data2.groupby(['Input.url']).label.nunique().eq(1).loc[url] == True:
        data2_unan_urls.append(url)
        if data2.loc[data2['Input.url']==url]['label'].iloc[0] =="Yes":
            data2_yes_urls.append(url)
        elif data2.loc[data2['Input.url']==url]['label'].iloc[0] =="No":
            data2_no_urls.append(url)
    if data3.groupby(['Input.url']).label.nunique().eq(1).loc[url] == True:
        data3_unan_urls.append(url)
        if data3.loc[data3['Input.url']==url]['label'].iloc[0] =="Yes":
            data3_yes_urls.append(url)
        elif data3.loc[data3['Input.url']==url]['label'].iloc[0] =="No":
            data3_no_urls.append(url)

data1_unanimous = data1[data1['Input.url'].isin(data1_unan_urls)]
# print(len(data1_unanimous))
data2_unanimous = data2[data2['Input.url'].isin(data2_unan_urls)]
# print(len(data2_unanimous))
data3_unanimous = data3[data3['Input.url'].isin(data3_unan_urls)]

a3 = data1_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a4 = data2_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a5 = data3_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
print("a3")
print(a3)

print("a4")
print(a4)

print("a5")
print(a5)

print("\nYES URLS")
print(len(data1_yes_urls))
print(len(data2_yes_urls))
print(len(data3_yes_urls))

print("\nNO URLS")
print(len(data1_no_urls))
print(len(data2_no_urls))
print(len(data3_no_urls))

# for url in data1['Input.url'].unique():
#     if data1.loc[data1['Input.url']==url, 'label'].value_counts().iloc[0] != data2.loc[data2['Input.url']==url, 'label'].value_counts().iloc[0]:
#         print(url)
#         print(data1.loc[data1['Input.url']==url, 'label'].value_counts())
#         print(data2.loc[data2['Input.url']==url, 'label'].value_counts())
#         print('\n')

# for url in data1['Input.url'].unique():
#     if data1.loc[data1['Input.url']==url, 'label'].value_counts().iloc[0] != data2.loc[data2['Input.url']==url, 'label'].value_counts().iloc[0]:
#         print(url)
#         print(data1.loc[data1['Input.url']==url, 'label'].value_counts())
#         print(data2.loc[data2['Input.url']==url, 'label'].value_counts())
#         print('\n')

# for i in data1.iterrows():
#     if i[1]['label'] != i[1]['Input.m_1']:
#         print(i)
#         break

# data1_unan = data1[["Input.url","label","Input.m_1"]]
# data2_unan = data2[["Input.url","label","Input.m_1"]]

# data1_unan = data1_unan[]


# data1_unan = data1[data1.groupby(['Input.url']).label.nunique().eq(1)]
# data2_unan = data2[data2.groupby(['Input.url']).label.nunique().eq(1)]


a3
True    78
Name: label, dtype: int64
a4
True    96
Name: label, dtype: int64
a5
True    50
Name: label, dtype: int64

YES URLS
53
85
48

NO URLS
25
11
2


In [72]:
# analyze the accuracy of the label agreed upon by the majority of labelers (majority voting)



# data1_majority_yes_urls = data1_yes_urls
# data2_majority_yes_urls = data2_yes_urls
# data3_majority_yes_urls = data3_yes_urls


data1_majority_yes_urls = [x for x in data1_yes_urls]
data2_majority_yes_urls = [x for x in data2_yes_urls]
data3_majority_yes_urls = [x for x in data3_yes_urls]

# data1_majority_no_urls = data1_no_urls
# data2_majority_no_urls = data2_no_urls
# data3_majority_no_urls = data3_no_urls

data1_majority_no_urls = [x for x in data1_no_urls]
data2_majority_no_urls = [x for x in data2_no_urls]
data3_majority_no_urls = [x for x in data3_no_urls]

print("NUMBER OF YES URLS")
print(len(data1_majority_yes_urls))

print("NUMBER OF NO URLS")
print(len(data1_majority_no_urls))
print("\n")

for url in data1['Input.url'].unique():
    if data1.groupby(['Input.url']).label.nunique().eq(2).loc[url] == True:
        # print(data1.loc[data1['Input.url']==url]['label'].value_counts())
        # print(data1.loc[data1['Input.url']==url]['label'].value_counts()[1])
        # print(data1.loc[data1['Input.url']==url]['label'].value_counts()[0])
        # break
        if data1.loc[data1['Input.url']==url]['label'].value_counts()['Yes'] > data1.loc[data1['Input.url']==url]['label'].value_counts()['No']:
            data1_majority_yes_urls.append(url)
        elif data1.loc[data1['Input.url']==url]['label'].value_counts()['Yes'] < data1.loc[data1['Input.url']==url]['label'].value_counts()['No']:
            data1_majority_no_urls.append(url)
    if data2.groupby(['Input.url']).label.nunique().eq(2).loc[url] == True:
        if data2.loc[data2['Input.url']==url]['label'].value_counts()['Yes'] > data2.loc[data2['Input.url']==url]['label'].value_counts()['No']:
            data2_majority_yes_urls.append(url)
        elif data2.loc[data2['Input.url']==url]['label'].value_counts()['Yes'] < data2.loc[data2['Input.url']==url]['label'].value_counts()['No']:
            data2_majority_no_urls.append(url)
    if data3.groupby(['Input.url']).label.nunique().eq(2).loc[url] == True:
        if data3.loc[data3['Input.url']==url]['label'].value_counts()['Yes'] > data3.loc[data3['Input.url']==url]['label'].value_counts()['No']:
            data3_majority_yes_urls.append(url)
        elif data3.loc[data3['Input.url']==url]['label'].value_counts()['Yes'] < data3.loc[data3['Input.url']==url]['label'].value_counts()['No']:
            data3_majority_no_urls.append(url)

print("NUMBER OF YES URLS")
print(len(data1_majority_yes_urls))

print("NUMBER OF NO URLS")
print(len(data1_majority_no_urls))
print("\n")

data1_majority_yes = data1.loc[data1['Input.url'].isin(data1_majority_yes_urls)]
data1_majority_no = data1.loc[data1['Input.url'].isin(data1_majority_no_urls)]

data2_majority_yes = data2.loc[data2['Input.url'].isin(data2_majority_yes_urls)]
data2_majority_no = data2.loc[data2['Input.url'].isin(data2_majority_no_urls)]

data3_majority_yes = data3.loc[data3['Input.url'].isin(data3_majority_yes_urls)]
data3_majority_no = data3.loc[data3['Input.url'].isin(data3_majority_no_urls)]

a1_yes = data1_majority_yes['correct_label'].value_counts()
a1_no = data1_majority_no['correct_label'].value_counts()
print(a1_yes)
print("TOTAL YES: " + str(len(data1_majority_yes)) + "\n")
print(a1_no)
print("TOTAL NO: " + str(len(data1_majority_no)) + "\n")
print("TOTAL: " + str(len(data1_majority_yes) + len(data1_majority_no)))
print(a1_yes[1]/len(data1_majority_yes))
print(a1_no[1]/len(data1_majority_no))
print("\n\n")

a2_yes = data2_majority_yes['correct_label'].value_counts()
a2_no = data2_majority_no['correct_label'].value_counts()
print(a2_yes)
print("TOTAL YES: " + str(len(data2_majority_yes)) + "\n")
print(a2_no)
print("TOTAL NO: " + str(len(data2_majority_no)) + "\n")
print("TOTAL: " + str(len(data2_majority_yes) + len(data2_majority_no)))
print(a2_yes[1]/len(data2_majority_yes))
print(a2_no[1]/len(data2_majority_no))
print("\n\n")

a3_yes = data3_majority_yes['correct_label'].value_counts()
a3_no = data3_majority_no['correct_label'].value_counts()
print(a3_yes)
print("TOTAL YES: " + str(len(data3_majority_yes)) + "\n")
print(a3_no)
print("TOTAL NO: " + str(len(data3_majority_no)) + "\n")
print("TOTAL: " + str(len(data3_majority_yes) + len(data3_majority_no)))
print(a3_yes[1]/len(data3_majority_yes))
print(a3_no[1]/len(data3_majority_no))


# a2_yes = data4_majority_yes['correct_label'].value_counts()
# a2_no = data4_majority_no['correct_label'].value_counts()
# print(a4_yes)
# print(a4_no)
# print(a4_yes[1]/len(data4_majority_yes))
# print(a4_no[1]/len(data4_majority_no))


# data1_unanimous = data1[data1['Input.url'].isin(data1_majority_urls)]
# # print(len(data1_unanimous))
# data2_unanimous = data2[data2['Input.url'].isin(data2_majority_urls)]
# # print(len(data2_unanimous))
# data3_unanimous = data3[data3['Input.url'].isin(data3_majority_urls)]

# a3 = data1_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
# a4 = data2_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
# a5 = data3_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
# print("a3")
# print(a3)

# print("a4")
# print(a4)

# print("a5")
# print(a5)

NUMBER OF YES URLS
53
NUMBER OF NO URLS
25


NUMBER OF YES URLS
129
NUMBER OF NO URLS
86


True     207
False    180
Name: correct_label, dtype: int64
TOTAL YES: 387

True     189
False     72
Name: correct_label, dtype: int64
TOTAL NO: 261

TOTAL: 648
0.5348837209302325
0.7241379310344828



False    255
True     246
Name: correct_label, dtype: int64
TOTAL YES: 501

True     106
False     41
Name: correct_label, dtype: int64
TOTAL NO: 147

TOTAL: 648
0.49101796407185627
0.7210884353741497



True     449
False    346
Name: correct_label, dtype: int64
TOTAL YES: 795

True     185
False    100
Name: correct_label, dtype: int64
TOTAL NO: 285

TOTAL: 1080
0.5647798742138365
0.6491228070175439


In [60]:
print(set([data1_majority_yes_urls in data1_majority_no_urls]))
print(set([data1_majority_no_urls in data1_majority_yes_urls]))

{False}
{False}


In [66]:
for url in data1['Input.url'].loc[range(0,5)].unique():
        print(data1.loc[data1['Input.url']==url]['label'].value_counts())
        print(data1.loc[data1['Input.url']==url]['label'].value_counts()['Yes'])
        print(data1.loc[data1['Input.url']==url]['label'].value_counts()[1])
        print(data1.loc[data1['Input.url']==url]['label'].value_counts()[0])

No     2
Yes    1
Name: label, dtype: int64
1
1
2
Yes    2
No     1
Name: label, dtype: int64
2
1
2


In [46]:
print(data3_majority_yes['label'].value_counts())
print(data3_majority_no['label'].value_counts())

Yes    240
Name: label, dtype: int64
Yes    479
No     361
Name: label, dtype: int64


In [21]:
a1_no

True     231
False    183
Name: correct_label, dtype: int64

a3
True    78
Name: label, dtype: int64
a4
True    96
Name: label, dtype: int64
a5
True    50
Name: label, dtype: int64


In [21]:
# For the comments that all labelers agreed on, calculate the percentage of agreement with the ground truth lables
# Remember that three or five  different people labeled each unanimous comment in all datasets derived below. 
# For this reason, the percentage of agreement is the percentage of unanimous comments that were labeled correctly.

data1_unanimous['correct_label'] = (data1_unanimous['label']==data1_unanimous['Input.m_1'])
print(len(data1_unanimous) / data1_n_labellers_per_comment)
a = data1_unanimous['correct_label'].value_counts()
print(a)
print(a[1]/len(data1_unanimous))

data2_unanimous['correct_label'] = (data2_unanimous['label']==data2_unanimous['Input.m_1'])
print(len(data2_unanimous) / data2_n_labellers_per_comment)
b = data2_unanimous['correct_label'].value_counts()
print(b)
print(b[1]/len(data2_unanimous))

data3_unanimous['correct_label'] = (data3_unanimous['label']==data3_unanimous['Input.m_1'])
print(len(data3_unanimous) / data3_n_labellers_per_comment)
c = data3_unanimous['correct_label'].value_counts()
print(c)
print(c[1]/len(data3_unanimous))

78.0
True     165
False     69
Name: correct_label, dtype: int64
0.7051282051282052
96.0
True     174
False    114
Name: correct_label, dtype: int64
0.6041666666666666
50.0
True     205
False     45
Name: correct_label, dtype: int64
0.82


<ipython-input-21-38f66a6a725a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_unanimous['correct_label'] = (data1_unanimous['label']==data1_unanimous['Input.m_1'])
<ipython-input-21-38f66a6a725a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2_unanimous['correct_label'] = (data2_unanimous['label']==data2_unanimous['Input.m_1'])
<ipython-input-21-38f66a6a725a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe